In [1]:
import pandas as pd
import numpy as np

In [38]:
weather_balloon= pd.read_csv(r'C:\Users\kmebr\Documents\data_analytics_project_fall_2023\DAEN-Contrail-Preventers\data/under_40kft_clean.csv')
weather_balloon

,Unnamed: 0,REPRH,CALCRH,TEMP,uid,altitude_calculated
0,0,920.0,NaN,2820,1,-427.281614
1,1,730.0,NaN,2858,1,61.478484
2,2,690.0,NaN,2852,1,363.644579
3,3,510.0,NaN,2822,1,1945.043989
4,4,210.0,NaN,2831,1,2352.232520
...,...,...,...,...,...,...
3317775,5951081,NaN,717.0,2868,97020,3211.738957
3317776,5951082,NaN,732.0,2842,97020,4779.193602
3317777,5951083,NaN,964.0,2725,97020,8650.081556
3317778,5951084,NaN,588.0,2715,97020,9078.237084


In [39]:
weather_balloon.fillna('', inplace=True)


In [40]:
vals = []
for real, calc in zip(weather_balloon['REPRH'].values, weather_balloon['CALCRH'].values):
    if real != '':
        vals.append(real)
    else:
        vals.append(calc)


In [41]:
weather_balloon['rel_humidity'] = vals

In [3]:
waether_balloon= weather_balloon.dropna(how='all')

In [42]:
def kelvin_to_fahrenheit(kelvin):
    fahrenheit = (kelvin/10 - 273.15) * 9/5 + 32
    return fahrenheit

columns_to_convert = ['TEMP']#, 'PTEMP','VTEMP','VPTEMP']
for col in columns_to_convert:
    new_col_name = col + '(F)'
    weather_balloon[new_col_name] = weather_balloon[col].apply(kelvin_to_fahrenheit)
weather_balloon = weather_balloon.drop(columns=columns_to_convert)


In [5]:
def meters_to_feet(meters):
    feet = meters * 3.28084
    return feet
weather_balloon['CALCGPH' + '(Ft)'] = weather_balloon['CALCGPH'].apply(meters_to_feet)
weather_balloon['REPGPH'+'(Ft)']= weather_balloon['REPGPH'].apply(meters_to_feet)
weather_balloon['UWND'+'(Ft/sec)']= weather_balloon['UWND'].apply(meters_to_feet)
weather_balloon['VWND'+'(Ft/sec)']= weather_balloon['VWND'].apply(meters_to_feet)
weather_balloon=weather_balloon.drop(['CALCGPH','REPGPH','UWND','VWND'],axis=1)

In [6]:
def k_per_km_to_f_per_ft(k_per_km):
    fahrenheit = (k_per_km/10 - 273.15) * 9/5 + 32   
    feet = 3280.84
    f_per_ft = fahrenheit / feet
    
    return f_per_ft

weather_balloon['PTEMPGRAD (F/Ft)'] = weather_balloon['PTEMPGRAD'].apply(k_per_km_to_f_per_ft)
weather_balloon['TEMPGRAD (F/Ft)'] = weather_balloon['TEMPGRAD'].apply(k_per_km_to_f_per_ft)
    
weather_balloon=weather_balloon.drop(['PTEMPGRAD','TEMPGRAD'],axis=1)
    
   


In [43]:

# Define a function to convert RH from water to ice
def rh_water_to_ice(rh_water):
    try:
        rh_ice = rh_water / 1.54
    except:
        rh_ice = np.nan
    return rh_ice

columns_to_convert = {
    'REPRH': 'REPRH(ice)',
    'CALCRH': 'CALCRH(ice)',
    'rel_humidity': 'RH_ICE_USE'}

for old_col, new_col in columns_to_convert.items():
    weather_balloon[new_col] = weather_balloon[old_col].apply(rh_water_to_ice)
    



In [44]:
weather_balloon=weather_balloon.drop(['REPRH','CALCRH'],axis=1)

In [45]:
weather_balloon.columns


Index(['Unnamed: 0', 'uid', 'altitude_calculated', 'rel_humidity', 'TEMP(F)',
       'REPRH(ice)', 'CALCRH(ice)', 'RH_ICE_USE'],
      dtype='object')

In [46]:
weather_balloon = weather_balloon[['uid', 'altitude_calculated', 'RH_ICE_USE', 'TEMP(F)']]
weather_balloon.columns = ['uid_fk', 'ALTITUDE(FT)', 'RH_ICE', 'TEMP(F)']

In [47]:
weather_balloon.head()

,uid_fk,ALTITUDE(FT),RH_ICE,TEMP(F)
0,1,-427.281614,597.402597,47.93
1,1,61.478484,474.025974,54.77
2,1,363.644579,448.051948,53.69
3,1,1945.043989,331.168831,48.29
4,1,2352.232520,136.363636,49.91


In [48]:
meta = pd.read_csv(r'C:\Users\kmebr\Documents\data_analytics_project_fall_2023\DAEN-Contrail-Preventers\data\weather_balloon_metadata.csv')

In [49]:
meta.head()

,SITEID,YEAR,MONTH,DAY,HOUR,RELTIME,NUMLEV,PW,INVPRESS,INVHGT,...,LFCHGT,LNBPRESS,LNBHGT,LI,SI,KI,TTI,CAPE,CIN,uid
0,#USM00072402,1963,10,14,12,NaN,26,944.0,101100.0,157.0,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1
1,#USM00072402,1963,10,15,12,NaN,24,1104.0,99500.0,287.0,...,NaN,NaN,NaN,11.0,8.0,-2.0,37.0,NaN,NaN,2
2,#USM00072402,1963,10,16,12,NaN,21,837.0,99600.0,227.0,...,NaN,NaN,NaN,8.0,13.0,-17.0,24.0,NaN,NaN,3
3,#USM00072402,1963,10,17,12,NaN,20,882.0,100200.0,157.0,...,NaN,NaN,NaN,10.0,13.0,-11.0,28.0,NaN,NaN,4
4,#USM00072402,1963,10,18,12,NaN,23,1895.0,100400.0,167.0,...,NaN,NaN,NaN,8.0,4.0,21.0,44.0,NaN,NaN,5


In [50]:
final = pd.merge(left=weather_balloon, right=meta, left_on='uid_fk', right_on='uid', how='left')

In [51]:
final.head()

,uid_fk,ALTITUDE(FT),RH_ICE,TEMP(F),SITEID,YEAR,MONTH,DAY,HOUR,RELTIME,...,LFCHGT,LNBPRESS,LNBHGT,LI,SI,KI,TTI,CAPE,CIN,uid
0,1,-427.281614,597.402597,47.93,#USM00072402,1963,10,14,12,NaN,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1
1,1,61.478484,474.025974,54.77,#USM00072402,1963,10,14,12,NaN,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1
2,1,363.644579,448.051948,53.69,#USM00072402,1963,10,14,12,NaN,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1
3,1,1945.043989,331.168831,48.29,#USM00072402,1963,10,14,12,NaN,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1
4,1,2352.232520,136.363636,49.91,#USM00072402,1963,10,14,12,NaN,...,NaN,NaN,NaN,15.0,16.0,-18.0,21.0,NaN,NaN,1


In [52]:
dates = []
for y, m, d in zip(final['YEAR'], final['MONTH'], final['DAY']):
    dates.append(str(y)+'-'+str(m)+'-'+str(d))
final['DATE'] = dates

In [53]:
final.columns

Index(['uid_fk', 'ALTITUDE(FT)', 'RH_ICE', 'TEMP(F)', 'SITEID', 'YEAR',
       'MONTH', 'DAY', 'HOUR', 'RELTIME', 'NUMLEV', 'PW', 'INVPRESS', 'INVHGT',
       'INVTEMPDIF', 'MIXPRESS', 'MIXHGT', 'FRZPRESS', 'FRZHGT', 'LCLPRESS',
       'LCLHGT', 'LFCPRESS', 'LFCHGT', 'LNBPRESS', 'LNBHGT', 'LI', 'SI', 'KI',
       'TTI', 'CAPE', 'CIN', 'uid', 'DATE'],
      dtype='object')

In [54]:
final2 = final[['uid', 'DATE', 'HOUR', 'ALTITUDE(FT)', 'RH_ICE', 'TEMP(F)', 'SITEID']]

In [55]:
final2['SITE'] = final2['SITEID'].map({'#USM00072402': 'Wallops Island', '#USM00072403': 'Sterling', '#USM00072405': 'Washington/National'})

C:\Users\kmebr\AppData\Local\Temp\ipykernel_31676\3864670402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final2['SITE'] = final2['SITEID'].map({'#USM00072402': 'Wallops Island', '#USM00072403': 'Sterling', '#USM00072405': 'Washington/National'})


In [56]:
final2.drop(columns=['SITEID'], inplace=True)

C:\Users\kmebr\AppData\Local\Temp\ipykernel_31676\2696037033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final2.drop(columns=['SITEID'], inplace=True)


In [57]:
final2.columns = ['uid_fk', 'DATE', 'HOUR', 'ALTITUDE(FT)', 'RH_ICE', 'TEMP(F)', 'SITE']

In [58]:
final2.to_csv(r'C:\Users\kmebr\Documents\data_analytics_project_fall_2023\DAEN-Contrail-Preventers\data/igra_noaa_for_dash.csv')

In [10]:
csv_filename = 'measurements_IGRA(converted units).csv'
weather_balloon.to_csv(csv_filename, index=False)

In [61]:
df = final2[(final2['ALTITUDE(FT)']>34000) & (final2['ALTITUDE(FT)']<38000)]

In [62]:
df.to_csv(r'C:\Users\kmebr\Documents\data_analytics_project_fall_2023\DAEN-Contrail-Preventers\data/igra_noaa_for_pred_map.csv')

In [68]:
final2.sort_values(by='DATE')

,uid_fk,DATE,HOUR,ALTITUDE(FT),RH_ICE,TEMP(F),SITE
3161974,85347,1950-10-1,3,35763.369982,337.012987,-51.79,Washington/National
3161758,85347,1950-10-1,3,6144.363709,600.649351,57.65,Washington/National
3161760,85347,1950-10-1,3,6895.012192,512.337662,56.03,Washington/National
3161762,85347,1950-10-1,3,7507.701373,581.168831,53.33,Washington/National
3161764,85347,1950-10-1,3,7593.338648,538.311688,53.33,Washington/National
...,...,...,...,...,...,...,...
1408923,39824,2023-9-5,12,747.671641,439.610390,81.59,Wallops Island
1408921,39824,2023-9-5,12,469.208310,465.584416,79.61,Wallops Island
1408919,39824,2023-9-5,12,363.644579,481.818182,79.07,Wallops Island
1408943,39824,2023-9-5,12,3651.442207,505.844156,69.89,Wallops Island


In [70]:
df.sort_values(by='DATE')

,uid_fk,DATE,HOUR,ALTITUDE(FT),RH_ICE,TEMP(F),SITE
3161986,85347,1950-10-1,3,37748.554595,48.051948,-60.07,Washington/National
3161988,85347,1950-10-1,3,37806.518232,3.246753,-59.35,Washington/National
3161984,85347,1950-10-1,3,37642.794556,320.129870,-61.33,Washington/National
3161982,85347,1950-10-1,3,37622.281328,351.948052,-61.33,Washington/National
3161980,85347,1950-10-1,3,37480.122952,470.129870,-61.15,Washington/National
...,...,...,...,...,...,...,...
1408910,39823,2023-9-5,0,36714.046091,111.038961,-58.45,Wallops Island
1408909,39823,2023-9-5,0,35008.274565,166.883117,-54.31,Wallops Island
1408908,39823,2023-9-5,0,34982.716853,166.883117,-54.13,Wallops Island
1409059,39824,2023-9-5,12,37112.856505,38.311688,-56.47,Wallops Island


In [72]:
df[df['DATE']=='2023-2-22']

,uid_fk,DATE,HOUR,ALTITUDE(FT),RH_ICE,TEMP(F),SITE
1387853,39483,2023-2-22,12,35885.313596,240.909091,-62.23,Wallops Island
3126370,84967,2023-2-22,0,34637.954342,244.805195,-66.73,Sterling
3126371,84967,2023-2-22,0,34703.230255,250.000000,-67.09,Sterling
3126372,84967,2023-2-22,0,35865.253322,275.974026,-71.59,Sterling
3126373,84967,2023-2-22,0,35955.648005,269.480519,-71.77,Sterling
3126374,84967,2023-2-22,0,36476.783939,129.220779,-67.27,Sterling
3126375,84967,2023-2-22,0,36489.855685,131.168831,-67.27,Sterling
3126376,84967,2023-2-22,0,36905.076158,127.272727,-67.99,Sterling
3126377,84967,2023-2-22,0,37457.800664,20.779221,-61.15,Sterling
3126378,84967,2023-2-22,0,37601.784705,22.727273,-60.97,Sterling
